## Web Scraping (Valve games vs Blizzard games)

Tasks:
- Scrape game company's wikipedia for list of games
- Scrape each game's info
- Cleaning
- Save to JSON, pickle, csv

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
valve_url = 'https://en.wikipedia.org/wiki/List_of_Valve_games'
blizzard_url = 'https://en.wikipedia.org/wiki/List_of_Blizzard_Entertainment_games'

valve_html = requests.get(valve_url)
blizzard_html = requests.get(blizzard_url)

In [3]:
soup = bs(valve_html.content)

contents = soup.prettify()

#print(contents)

In [4]:
games = soup.select('.wikitable i a')
game_path = games[0]['href']

#game_path

In [5]:
def get_game_links(url):
    html = requests.get(url)
    soup = bs(html.content)
    games = soup.select('.wikitable i a')
    games = soup.select('.wikitable .fn i a')
    game_links = []
    base_path = 'https://en.wikipedia.org'

    for game in games:
        # subdirectory example: "/wiki/Half-Life_(video_game)"
        game_subdirectory = game['href']
        path = base_path + game_subdirectory
        game_links.append(path)
    
    return game_links

#get_game_links('https://en.wikipedia.org/wiki/List_of_Valve_games')

Cleaning covers:
- Remove references [a] [1] etc
- Convert from all li elements in one string to list of strings
- Remove whitespace chars like br, \n, xa0, etc
- Convert from release date string to datetime objects
- Remove random strings in list at the end like (text), '', ':'

In [6]:
import re

def remove_ref(contents):
    # regular expression to find all matches with format [c] where c is any character eg. [a] or [1]
    reference = r'\[.\]'
    
    if type(contents) == str:
        matches = re.findall(reference, contents)
        # for each match, remove it from the string
        for match in matches:
            contents = contents.replace(match, '')
        # final remove whitespace that might've been between ref n text
        contents = contents.strip()
        return contents
    
    elif type(contents) == list:
        contents_list = []
        for my_string in contents:
            matches = re.findall(reference, my_string)
            for match in matches:
                my_string = my_string.replace(match, '')
                # final remove whitespace that might've been between ref n text
                my_string = my_string.strip()
            contents_list.append(my_string)
        return contents_list

In [7]:
def get_row_contents(row):
    # to handle releases
    # strip=True to remove whitespace chars (\n, xa0, etc)
    if row.select('li ul li'):
        return [li.get_text('', strip=True) for li in row.select('li ul li')]
    elif row.find('li'):
        return [li.get_text('', strip=True) for li in row.find_all('li')]
    elif row.find('br'):
        # stripped strings strips out whitespace chars
        return [text for text in row.find('td').stripped_strings]
    elif "," in row.get_text():
        return [text.strip() for text in row.find('td', class_='infobox-data').get_text().split(',')]
    else:  
        #print(row)
        return row.find('td', class_='infobox-data').get_text()

In [8]:
import re

def standardise_date(date):
    # other date format
    uncommon_date_re = r'\d+[ ][A-Za-z]+[ ]\d+'
    if re.search(uncommon_date_re, date):
        # first split string to list
        # eg. 15 June 2005 to [15, 'June', 2005]
        split_date = date.split()
        # make date string like this: June 15, 2005
        new_date = f'{split_date[1]} {split_date[0]}, {split_date[2]}'
        return new_date
    else:
        return date

standardise_date('15 June 2005')

'June 15, 2005'

In [9]:
from datetime import datetime

def convert_to_datetime(date_str):
    date_format = '%B %d, %Y'
    date = datetime.strptime(date_str, date_format)
    return date

convert_to_datetime('June 15, 2005')

datetime.datetime(2005, 6, 15, 0, 0)

In [10]:
import re

def get_release_date(row):
    if row.select('ul li'):
        contents = [li.get_text('', strip=True) for li in row.select('ul li')]
        #print('in ul li if statement')
    else:
        contents = row.find('td', class_='infobox-data').get_text()
        #print('in everything else')
    #print(contents)
    # June 15, 2005 or 15 June 2005 
    date_re = r'([A-Za-z]+[ ]\d+,[ ]\d+)|(\d+[ ][A-Za-z]+[ ]\d+)'
    
    if type(contents) == str:
        match = re.search(date_re, contents)
        date = match.group()
        standardised_date = standardise_date(date)
        datetime_obj = convert_to_datetime(standardised_date)
        return datetime_obj
    elif type(contents) == list:
        for i, item in enumerate(contents):
            match = re.search(date_re, item)
            #print(i, match)
            if (match != None):
                date = match.group()
                standardised_date = standardise_date(date)
                datetime_obj = convert_to_datetime(standardised_date)
                return datetime_obj
        return None

#get_release_date(['Microsoft Windows', '29 November 2006', 'Mac OS X', '23 September 2010', 'Linux', '5 June 2013'])

#### Scrape game ratings from Metacritic

In [11]:
import re

def name_to_link(name):
    # matches when a char is not a-zA-Z or 0-9 or "_"
    alphanumeric_re = r'\W'
    
    name = name.lower()
    name = name.replace(' ', '_')
    name = name.replace('-', '_')
    # remove matches of non alphanumeric or "_"
    name = re.sub(alphanumeric_re, '', name)
    name = name.replace('_', '-')
    
    return name

#print(name_to_link('Garry\'s Mod'))
#print(name_to_link('Half life 2: Lost Coast'))

In [12]:
def get_game_rating(name):
    # change name to link format
    # eg. https://www.metacritic.com/game/pc/garrys-mod, https://www.metacritic.com/game/pc/half-life
    name = name_to_link(name)
    url = f'https://www.metacritic.com/game/pc/{name}'

    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)
    soup = bs(response.content)
    
    metascore = soup.find('span', itemprop='ratingValue')
    if metascore is not None:
        metascore_str = metascore.get_text()
    else:
        metascore_str = 'N/A'
        
    userscore = soup.select('.score_summary .metascore_anchor .metascore_w.user')
    # by default returns empty list
    if len(userscore) > 0:
        userscore_str = userscore[0].get_text()
    else:
        userscore_str = 'N/A'
    
    return metascore_str, userscore_str

In [13]:
# deal with random (text) and '' whitespaces at the end
def remove_random_contents(contents):
    # to match format (Japan something) or ':'
    random_re = r'(\(*\))|[:]'
    if type(contents) == list:
        for item in contents:
            # if find item in format (Japan something) or ':' or empty string ''
            if re.search(random_re, item) or len(item) == 0:
                # remove that item from contents
                contents.remove(item)
    return contents

In [14]:
def get_info_box(url):
    game_html = requests.get(url)
    game_soup = bs(game_html.content)
    base_path = 'https://en.wikipedia.org'
    
    info_box = game_soup.find('table', class_='infobox')
    info_rows = info_box.find_all('tr')
    
    game_info = {}
    
    for i, row in enumerate(info_rows):
        # get title from infobox
        if row.select('.infobox-above'):
            game_info['Name'] = row.find('th').get_text()
            # handling really rare edge case with different naming
            if game_info['Name'] == 'Hearthstone':
                game_info['Name'] = 'Hearthstone: Heroes of Warcraft'
        # get image link from infobox
        elif row.select('.infobox-image'):
            image_link = base_path + row.find('a')['href']
            game_info['Image_link'] = image_link
        else:
            label = row.find('th', class_='infobox-label').get_text()
            contents = get_row_contents(row)
            contents = remove_ref(contents)
            if (label == 'Release'):
                #print(contents)
                contents = get_release_date(row)
                #print(contents)
            #print(contents)
            contents = remove_random_contents(contents)
            #print(contents)
            #print(f'Label: {label}, type: {type(contents)}')
                
            #print(contents)
            game_info[label] = contents
    
    # if no Release label found, it's not yet released
    if 'Release' not in game_info:
        game_info['Release'] = None
    
    game_info['Source'] = url
    
    metascore, userscore = get_game_rating(game_info['Name'])
    game_info['Metascore'] = metascore
    game_info['User Score'] = userscore
    
    return game_info
    
#get_info_box('https://en.wikipedia.org/wiki/Justice_League_Task_Force_(video_game)')

In [15]:
def scrape_game_company(url):
    game_links = get_game_links(url)
    games_info = []
    for i, link in enumerate(game_links):
        try:
            games_info.append(get_info_box(link))
            if (i % 10 == 0):
                print(f'--Scraped {i} games--')

        except Exception as e:
            print(link)
            print(e)

    print(f'Scraping finished with {i} games')
    return games_info

In [16]:
valve_games = scrape_game_company(valve_url)
#valve_games

--Scraped 0 games--
--Scraped 10 games--
--Scraped 20 games--
Scraping finished with 26 games


In [17]:
# ignore exceptions (they're games that haven't been released)
blizzard_games = scrape_game_company(blizzard_url)
#blizzard_games

--Scraped 0 games--
--Scraped 10 games--
--Scraped 20 games--
Scraping finished with 21 games


In [18]:
# see random game scraped
blizzard_games[10]

{'Name': 'StarCraft',
 'Image_link': 'https://en.wikipedia.org/wiki/File:StarCraft_box_art.jpg',
 'Developer(s)': 'Blizzard Entertainment',
 'Publisher(s)': 'Blizzard Entertainment',
 'Designer(s)': ['Chris Metzen', 'James Phinney'],
 'Composer(s)': ['Derek Duke',
  'Jason Hayes',
  'Glenn Stafford',
  'Tracy W. Bush'],
 'Series': 'StarCraft',
 'Platform(s)': ['Microsoft Windows', 'Classic Mac OS', 'Nintendo 64'],
 'Release': datetime.datetime(1998, 3, 31, 0, 0),
 'Genre(s)': 'Real-time strategy',
 'Mode(s)': ['Single-player', 'multiplayer'],
 'Source': 'https://en.wikipedia.org/wiki/StarCraft_(video_game)',
 'Metascore': '88',
 'User Score': '9.1'}

#### Save games to different file formats

In [19]:
from datetime import datetime

# converts datetime objects in game dictionaries to strings
def json_friendly(game_dict):
    for game in game_dict:
        datetime_obj = game['Release']
        if datetime_obj == None:
            continue
        date_str = datetime_obj.strftime('%B %d, %Y')
        game['Release'] = date_str
    return game_dict
    
#a = json_friendly(blizzard_games)

In [20]:
# Save and load JSON files - compatible with most programming languages
import json

def save_json(filename, data):
    data = json_friendly(data)
    with open(filename, 'w') as f:
        json.dump(data, f)
        
def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [21]:
# Save and load pickle files - can store python specific objects like datetime
import pickle

def save_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [22]:
save_pickle('valve_cleaned_pickle.pickle', valve_games)
save_pickle('blizzard_cleaned_pickle.pickle', blizzard_games)

In [23]:
save_json('valve_cleaned_json.json', valve_games)
save_json('blizzard_cleaned_json.json', blizzard_games)

In [24]:
import pandas as pd

valve_df = pd.DataFrame.from_dict(valve_games)
blizzard_df = pd.DataFrame.from_dict(blizzard_games)

valve_df.to_csv('valve_cleaned_csv.csv', index=False)
blizzard_df.to_csv('blizzard_cleaned_csv.csv', index=False)

#### Try loading data in

In [25]:
valve_json = load_json('valve_cleaned_json.json')
blizzard_json = load_json('blizzard_cleaned_json.json')

In [26]:
valve_games = load_pickle('valve_cleaned_pickle.pickle')
blizzard_games = load_pickle('blizzard_cleaned_pickle.pickle')